# Playground

This is a space for experimentation.

In [1]:
import random
import itertools
import numpy as np
from player import Player
from manual import Manual
from minimax import MiniMax
from strategy import Strategy
from datetime import datetime
from tic_tac_toe import WorldTTT
from q_learning import TabQLearning
# from utility import state_str_to_num
# from utility import state_num_to_str
from utility import list_to_tuple_2d
from utility import get_opposite_symbol

%load_ext autoreload
%autoreload 2

In [2]:
world_ttt = WorldTTT()

In [13]:
s = list_to_tuple_2d([
    ['#', '#', '#'],
    ['#', '#', '#'],
    ['#', '#', 'X']
])

In [10]:
strategy_tabq = TabQLearning(
    is_game_over=world_ttt.is_game_over,
    get_next_states=world_ttt.get_next_states,
    get_next_state=world_ttt.get_next_state,
    states=world_ttt.states,
    actions=world_ttt.actions,
    r_tab=world_ttt.get_reward_table(),
)
r_tab = np.array(strategy_tabq.r_tab)

In [8]:
world_ttt.actions.index((1, 1, 'O'))

9

In [7]:
world_ttt.get_next_state(s, (1, 1, 'X'))

(False, None)

In [6]:
r_tab[world_ttt.states.index(s), :]

array([-100., -100., -100.,   -5., -100., -100., -100.,   -5., -100.,
         -5., -100.,   -5., -100., -100., -100., -100., -100., -100.])

In [11]:
strategy_tabq.learn(
    symbols=['X', 'O'], 
    max_episodes=1e6, 
    # change_threshold=0,
    discount_factor=0.9,
    learning_rate=0.3,
    # start_choice_threshold={
    #     'state_count':len(world_ttt.states),
    #     'choice_count': 100
    # }
)

Learning ...
Max episodes reached.
All done. Episodes = 100000.


In [12]:
strategy_tabq.save_qtab("gamma0.9alpha0.3episodes1e6", "./q_tables")

Saved Q table at ./q_tables/gamma0.9alpha0.1episodes1e6.json.


In [66]:
world_ttt.actions.index((1, 1, 'X'))

8

In [70]:
world_ttt.states.index(s)

3107

In [71]:
strategy_tabq.r_tab[world_ttt.states.index(s)]

[-100,
 -100,
 -100,
 -5.0,
 -100,
 -100,
 -100,
 -5.0,
 -100,
 -5.0,
 -100,
 -5.0,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100]

In [65]:
print(strategy_tabq.q_tab[world_ttt.states.index(s), :])

[-1000 -1000  -100    -5 -1000 -1000  -100    -5  -100    -5  -100    -5
 -1000 -1000 -1000 -1000 -1000 -1000]


In [60]:
print(strategy_tabq.q_tab[world_ttt.states.index(s), :])

[-1000 -1000  -100    -5 -1000 -1000  -100    -5  -100    -5  -100    -5
 -1000 -1000 -1000 -1000 -1000 -1000]


In [61]:
move_idx = strategy_tabq.get_move(state=s, sym='O')['f_out']
print(f"Move Index = {move_idx}")
print(f"Move = {world_ttt.actions[move_idx]}")

Move Index = 3
Move = (0, 1, 'O')
